In [3]:
from google.colab import drive
drive.mount('/content/drive')

!pip install deepchem

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached deepchem-2.6.1-py3-none-any.whl (608 kB)
  Using cached rdkit_pypi-2022.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.8 MB)


In [4]:
import glob,os

import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf

In [5]:
DATASET_FILE ='/content/drive/My Drive/DL/20220904分类/数据库.csv'
MODEL_DIR = '/content/drive/My Drive/DL/20220904分类/GNN_model'

In [16]:
# ConvMolFeaturizer()
from sklearn.metrics import roc_auc_score

featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.CSVLoader(tasks=["Tg/℃"], feature_field="Smiles", featurizer=featurizer)
dataset = loader.create_dataset(DATASET_FILE, shard_size=1000)

splitter = dc.splits.splitters.RandomSplitter()
trainset, testset = splitter.train_test_split(dataset, frac_train=0.8,seed=1)

model = dc.models.GraphConvModel(
    1, mode="classification", batch_normalize=False, batch_size=10,model_dir=MODEL_DIR,n_classes=3, dropout=0.2)
# model = dc.models.DAGModel(1, mode="regression", batch_normalize=False, batch_size=10,model_dir=MODEL_DIR, dropout=0.2)

model.fit(trainset, nb_epoch=1000)

test_pred = model.predict(testset)

pred0 = np.argmax(np.squeeze(test_pred),axis=1)
pred0
ba0 = metrics.balanced_accuracy_score(y_true=testset.y, y_pred=pred0)
ba0

    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
/usr/local/lib/python3.7/dist-packages/deepchem/feat/base_classes.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(features)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_11/Reshape_14:0", shape=(87,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_11/Reshape_13:0", shape=(87, 64), dtype=float32), dense_shape=Tensor("gradient_tape/pr

0.8482362082362083